In [ ]:
# clean text
from textblob import TextBlob
import re
import string


def decontracted(phrase):

    # Specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # ..

    # General
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    # ..

    return phrase

def remove_punctuations(text):
    for punctuation in list(string.punctuation): text = text.replace(punctuation, '')
    return text

def clean_number(text):
    text = re.sub(r'(\d+)([a-zA-Z])', '\g<1> \g<2>', text)
    text = re.sub(r'(\d+) (th|st|nd|rd) ', '\g<1>\g<2> ', text)
    text = re.sub(r'(\d+),(\d+)', '\g<1>\g<2>', text)
    return text

def clean_whitespace(text):
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text

def clean_repeat_words(text):
    return re.sub(r"(\w*)(\w)\2(\w*)", r"\1\2\3", text)

def clean_text(text):
    # text_blob = TextBlob(text)
    # text = str(text_blob.correct())
    text = str(text)
    text = decontracted(text)
    text = remove_punctuations(text)
    text = clean_number(text)
    text = clean_whitespace(text)
    
    return text

In [ ]:
import torch
import pandas as pd
from dataset import AutoTokenizer, LANGUAGE_TOKENS, CATEGORY_TOKENS, LEVEL_TOKENS, KIND_TOKENS, OTHER_TOKENS
from model import Model

from torch.utils.data import DataLoader, Dataset, default_collate

In [ ]:
from pathlib import Path


TEST_MODE = False

# --------------------- VALIDATION SET --------------------------
from tqdm import tqdm
if not TEST_MODE:
    data_df = pd.read_csv("./data/siamese_train.csv")
    fold = 0

data_folder = Path("./data")
# TODO: we have to process for test set ourselves
contents_df = pd.read_csv(data_folder/'content.csv')
contents_df = contents_df.fillna('')
contents_df['title_len'] = contents_df.title.str.len()
contents_df = contents_df.sort_values(by='title_len', axis=0).reset_index(drop=True).drop(columns=['title_len'])
topics_df = pd.read_csv(data_folder/'topics.csv')
topics_df = topics_df.fillna('')
topics_df['title_len'] = topics_df.title.str.len()
topics_df = topics_df.sort_values(by='title_len', axis=0).reset_index(drop=True).drop(columns=['title_len'])
subs_df = pd.read_csv(data_folder/'sample_submission.csv')
corrs_df = pd.read_csv(data_folder/'correlations.csv')


topics_df["title"] = topics_df["title"].apply(clean_text)
topics_df["description"] = topics_df["description"].apply(clean_text)

contents_df["title"] = contents_df["title"].apply(clean_text)
contents_df["description"] = contents_df["description"].apply(clean_text)
contents_df["text"] = contents_df["text"].apply(clean_text)

In [ ]:
from tqdm import tqdm

if TEST_MODE:
    topics_df = topics_df[topics_df.id.isin(subs_df.topic_id)]
else: # VAL_MODE
    topics_df = topics_df[topics_df.id.isin(data_df[data_df["fold"] == fold].topics_ids)]

topics_df["topic_text"] = "<|topic|>" + f"<|lang_{topics_df['language']}|>" + f"<|category_{topics_df['category']}|>" + f"<|level_{topics_df['level']}|>" + "<s_title>" + topics_df["title"] + "</s_title>" + "<s_description>" + topics_df["description"] + "</s_description>"
topics_df["topic_text"] = topics_df["topic_text"].apply(lambda x: x[:2048])

contents_df["content_text"] =  "<|content|>" + f"<|lang_{contents_df['language']}|>" + f"<|kind_{contents_df['kind']}|>" + "<s_title>" + contents_df["title"] + "</s_title>" + "<s_description>" + contents_df["description"] + "</s_description>" + "<s_text>" + contents_df["text"] + "</s_text>"
contents_df["content_text"] = contents_df["content_text"].apply(lambda x: x[:2048])

In [ ]:
class InferenceDataset(Dataset):
    def __init__(self, texts, tokenizer_name='xlm-roberta-base', max_len=512):
        self.texts = texts

        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.tokenizer.add_special_tokens(dict(additional_special_tokens=LANGUAGE_TOKENS + CATEGORY_TOKENS + LEVEL_TOKENS + KIND_TOKENS + OTHER_TOKENS))
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        
        # topic
        inputs = self.tokenizer.encode_plus(
            text, 
            return_tensors = None, 
            add_special_tokens = True, 
            max_length = self.max_len,
            pad_to_max_length = True,
            truncation = True
        )
        for k, v in inputs.items():
            inputs[k] = torch.tensor(v, dtype = torch.long)
            
        return inputs
    
def collate_fn(inputs):
    inputs = default_collate(inputs)
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
        
    return inputs

In [ ]:
topic_dataset = InferenceDataset(texts=list(topics_df.topic_text.values))
topic_dataloader = DataLoader(topic_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)



In [ ]:
weights_path = "xlm-roberta-base"

model = Model(tokenizer=topic_dataset.tokenizer, model_name=weights_path)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [ ]:
topic_embs = []

for inputs in tqdm(topic_dataloader):
    out = model.feature(inputs)
    topic_embs.extend(out.detach().numpy())


In [ ]:
content_dataset = InferenceDataset(texts=list(content_df.content_text.values))
content_dataloader = DataLoader(content_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

content_embs = []

for inputs in tqdm(content_dataloader):
    out = model.feature(inputs)
    content_embs.extend(out.detach().numpy())


In [ ]:
!pip install annoy==1.0.3

In [ ]:
from fuzzywuzzy import fuzz, process

from annoy import AnnoyIndex


content_forest = AnnoyIndex(content_embs.shape[1], metric='angular')
for i, item in tqdm(enumerate(content_embs), total=len(content_embs)):
    content_forest.add_item(i, item)
content_forest.build(100)

In [ ]:
nearest_content_count = 400
fuzzy_filter = 86

topics = topics_df[topics_df.has_content==True][['id', 'title', 'language']].reset_index(drop=True)

test = topics
all_content_ids = contents_df.id.to_numpy()
all_content_titles = contents_df.title.to_numpy()
all_content_language = contents_df.language.to_numpy()
all_test_ids = list(topics.id)
all_test_title = list(topics.title)
all_test_language = list(test.language)

preds = []
for i, t_e in tqdm(enumerate(topic_embs), total=len(topic_embs), desc=f'Getting Preds'):
    indexes = content_forest.get_nns_by_vector(t_e, nearest_content_count)
    topic_id = all_test_ids[i]
    topic_text = all_test_title[i]
    topic_lang = all_test_language[i]
    
    filtered_indexes = []
    for idx in indexes:
        if topic_lang != all_content_language[idx]:
            continue
        fuzzy_value = fuzz.token_set_ratio(all_content_titles[idx], topic_text)
        if fuzzy_value > 86:
            filtered_indexes.append(idx)

    if len(filtered_indexes) == 0:
        indexes = list(set(filtered_indexes + indexes[:8-len(filtered_indexes)]))
    else:
        indexes = filtered_indexes[:10]
    content_ids = all_content_ids[indexes]
    preds.append({
        'topic_id': topic_id,
        'content_ids': ' '.join(content_ids)
    })
preds = pd.DataFrame.from_records(preds)

preds.to_csv('submission.csv', index=False)

In [ ]:
if not TEST_MODE:
    from engine import f2_score
    gt = corrs_df[corrs_df.topic_id.isin(data_df[(data_df["fold"] == fold)].topics_ids)].sort_values("topic_id")    
    preds = preds.sort_values("topic_id")    
    print("f2_score", f2_score(gt["content_ids"], preds["content_ids"]))